# Pretrain model in the HRRS dataset

This notebook is used to pretrain the backbone model using the HRRS dataset.

## 1. Import modules and define parameters

In [ ]:
# clone repository with the backbone model
!git clone https://github.com/LucasKirsten/InceptionV4_TensorFlow2.git

In [ ]:
# ignore all future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# import necessary modules
import sys
sys.path.append('./InceptionV4_TensorFlow2')
from inception_resnet_v2 import InceptionResNetV2

import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as KL
from tqdm import tqdm
from glob import glob

In [ ]:
# use mixed precision training (i.e., variables in float16 and computation in float32)
# this speeds up the training and allows higher batch sizes
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
# define strategy
if len(tf.config.experimental.list_physical_devices('GPU'))>1:
    strategy = tf.distribute.MirroredStrategy()
    print('Using mirrored strategy')
else:
    strategy = tf.distribute.get_strategy()

In [ ]:
# define the parameters for training
BATCH_SIZE = 64
PATH_TFRECORDS = './HRRS_tfrecords/*.tfrec'
LR = 0.1*np.sqrt(BATCH_SIZE/256)
LR_ATTENUATION = 0.1
TOTAL_STEPS = int(100000)
STEPS_PER_EPOCH = int(25000)
EPOCHS = int(TOTAL_STEPS/STEPS_PER_EPOCH)

## 2. Define DL model

In [ ]:
# define the DL model
with strategy.scope():
    inputs = KL.Input((256,256,3))
    backbone = InceptionResNetV2()
    x = backbone(inputs)
    x = KL.GlobalAveragePooling2D() (x)
    x = KL.Dense(92, activation='softmax') (x)

    model = tf.keras.Model(inputs, x)

model.summary()

## 3. Define data loader

In [ ]:
# function to read data from tfrecords
def __read_data(example):
    LABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = tf.reshape(tf.io.decode_raw(example['image'], tf.uint8), (256,256,3))
    image = tf.cast(image, 'float32')/255.
    label = tf.reshape(tf.io.decode_raw(example['label'], tf.float32), (92,))
    return image, label


ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

dataset = tf.data.TFRecordDataset(tf.io.gfile.glob(PATH_TFRECORDS),\
                                  compression_type='GZIP',\
                                  num_parallel_reads=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(2048)
dataset = dataset.with_options(ignore_order)
dataset = dataset.map(__read_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
dataset = dataset.repeat()

## 4. Define training setup

In [ ]:
# learning rate scheduler
def _schedule(epoch, lr):
    if epoch>1:
        return lr*float(LR_ATTENUATION)
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(_schedule, verbose=1)

In [ ]:
# compile model
with strategy.scope():
    model.compile(tf.keras.optimizers.SGD(learning_rate=LR), \
                  loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# train model
history = model.fit(
    dataset,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    callbacks = [lr_scheduler]
)